# Time series clustering exercise

In [1]:
from __future__ import print_function 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import geopandas as gpd
from fiona.crs import from_epsg
import shapely
import urllib.request
import requests
import json
from io import StringIO
import io

# Data

Download the census bureau business data for all years 1993-2014. You can investigate using the API (I have not done it with the census bureau). I did is as you see below

In [2]:
#these commands can be run on the shell and get the data with the command wget
#the cell needs to be run only once
!for ((y=94; y<=99; y+=1)); do wget \
https://www2.census.gov/Econ2001_And_Earlier/CBP_CSV//zbp$y\totals.zip; mv \
zbp$y\totals.zip $PUIDATA; done

!for ((y=0; y<=1; y+=1)); do wget \
https://www2.census.gov/Econ2001_And_Earlier/CBP_CSV//zbp0$y\totals.zip; mv \
zbp0$y\totals.zip $PUIDATA; done

!for ((y=2; y<=9; y+=1)); do wget \
https://www2.census.gov/econ200$y\/CBP_CSV/zbp0$y\totals.zip; mv \
zbp0$y\totals.zip $PUIDATA; done

!for ((y=10; y<=14; y+=1)); do wget \
https://www2.census.gov/econ20$y\/CBP_CSV/zbp$y\totals.zip; mv \
zbp$y\totals.zip $PUIDATA; done

--2017-12-12 01:39:05--  https://www2.census.gov/Econ2001_And_Earlier/CBP_CSV//zbp94totals.zip
Resolving www2.census.gov... 104.99.148.110, 2600:1408:1d:38b::208c, 2600:1408:1d:386::208c
Connecting to www2.census.gov|104.99.148.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611524 (597K) [application/zip]
Saving to: 'zbp94totals.zip'

zbp94totals.zip     100%[===================>] 597.19K  --.-KB/s    in 0.1s    

2017-12-12 01:39:05 (5.95 MB/s) - 'zbp94totals.zip' saved [611524/611524]

--2017-12-12 01:39:05--  https://www2.census.gov/Econ2001_And_Earlier/CBP_CSV//zbp95totals.zip
Resolving www2.census.gov... 104.99.148.110, 2600:1408:1d:38b::208c, 2600:1408:1d:386::208c
Connecting to www2.census.gov|104.99.148.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625328 (611K) [application/zip]
Saving to: 'zbp95totals.zip'

zbp95totals.zip     100%[===================>] 610.67K  --.-KB/s    in 0.1s    

2017-12-12 01:39:06 (6.10 M

zbp09totals.zip     100%[===================>] 644.64K  --.-KB/s    in 0.1s    

2017-12-12 01:39:08 (5.61 MB/s) - 'zbp09totals.zip' saved [660109/660109]

--2017-12-12 01:39:09--  https://www2.census.gov/econ2010/CBP_CSV/zbp10totals.zip
Resolving www2.census.gov... 104.99.148.110, 2600:1408:1d:38b::208c, 2600:1408:1d:386::208c
Connecting to www2.census.gov|104.99.148.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 797534 (779K) [application/zip]
Saving to: 'zbp10totals.zip'

zbp10totals.zip     100%[===================>] 778.84K  --.-KB/s    in 0.1s    

2017-12-12 01:39:09 (6.16 MB/s) - 'zbp10totals.zip' saved [797534/797534]

--2017-12-12 01:39:09--  https://www2.census.gov/econ2011/CBP_CSV/zbp11totals.zip
Resolving www2.census.gov... 104.99.148.110, 2600:1408:1d:38b::208c, 2600:1408:1d:386::208c
Connecting to www2.census.gov|104.99.148.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 795390 (777K) [application/zip]
Saving t

 Download the NYC zipcodes shapefile. One of many ways in which you can get the zipcodes shapefile for NYC
 https://data.cityofnewyork.us/download/i8iw-xf4u/application%2Fzip


In [3]:
urllib.request.urlretrieve('https://data.cityofnewyork.us/download/i8iw-xf4u/application%2Fzip', "file.gz")

('file.gz', <http.client.HTTPMessage at 0x10b6d8cf8>)

In [4]:
os.system("mv " + "file.gz " + os.getenv("PUIDATA"))
os.system("unzip " + os.getenv("PUIDATA") + "/file.gz -d " + os.getenv("PUIDATA") + "/zipcode")
nyc = gpd.read_file(os.getenv("PUIDATA") + "/zipcode" + "/ZIP_CODE_040114.shp")
nyc.drop(['BLDGZIP', 'PO_NAME', 'POPULATION', 'AREA', 'STATE', 'COUNTY', 'ST_FIPS', 'CTY_FIPS', 'URL'], axis=1, inplace=True)
nyc.head()

,ZIPCODE,SHAPE_AREA,SHAPE_LEN,geometry
0,11436,0.0,0.0,"POLYGON ((1038098.251871482 188138.3800067157,..."
1,11213,0.0,0.0,"POLYGON ((1001613.712964058 186926.4395172149,..."
2,11212,0.0,0.0,"POLYGON ((1011174.275535807 183696.33770971, 1..."
3,11225,0.0,0.0,"POLYGON ((995908.3654508889 183617.6128015518,..."
4,11218,0.0,0.0,"POLYGON ((991997.1134308875 176307.4958601296,..."


## You can use zipfile module in python to unzip the files
it should be install in your system, but if it is not you can get the code with wget from here
https://github.com/python/cpython/blob/2.7/Lib/zipfile.py
remembering to use the raw link
(or you can use the usual shell commands, and miss the chance to learn something new)

In [5]:
!wget https://raw.githubusercontent.com/python/cpython/2.7/Lib/zipfile.py

--2017-12-12 01:39:12--  https://raw.githubusercontent.com/python/cpython/2.7/Lib/zipfile.py
Resolving raw.githubusercontent.com... 151.101.208.133
Connecting to raw.githubusercontent.com|151.101.208.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15060 (15K) [text/plain]
Saving to: 'zipfile.py.2'

zipfile.py.2        100%[===================>]  14.71K  --.-KB/s    in 0.009s  

2017-12-12 01:39:12 (1.54 MB/s) - 'zipfile.py.2' saved [58813]



In [6]:
import pandas as pd
import zipfile 

In [11]:
fname_list = []
for i in range(94, 100):
    name = 'zbp' + str(i) + 'totals.zip'
    fname_list.append(name)
for i in range(0, 10):
    name = 'zbp0' + str(i) + 'totals.zip'
    fname_list.append(name)
for i in range(10, 15):
    name = 'zbp' + str(i) + 'totals.zip'
    fname_list.append(name)

In [9]:
for fname in fname_list:
    zf = zipfile.ZipFile(os.getenv("PUIDATA") + "/" + fname)
    df = pd.read_csv(zf.open(fname.replace('.zip','.txt')))
    df.rename(str.lower, axis='columns', inplace=True)
    #print(df.columns)
    df['zip'] = df['zip'].astype(str)
    df = df[['zip', 'est']]
    df.rename(columns={'est': fname[:5]}, inplace=True)
    nyc = nyc.merge(df, left_on='ZIPCODE', right_on='zip')
    nyc.drop(['zip'], axis=1, inplace=True)
    print(fname, 'done')

zbp94totals.zip done
zbp95totals.zip done
zbp96totals.zip done
zbp97totals.zip done
zbp98totals.zip done
zbp99totals.zip done
zbp00totals.zip done
zbp01totals.zip done
zbp02totals.zip done
zbp03totals.zip done
zbp04totals.zip done
zbp05totals.zip done
zbp06totals.zip done
zbp07totals.zip done
zbp08totals.zip done
zbp09totals.zip done
zbp10totals.zip done
zbp11totals.zip done
zbp12totals.zip done
zbp13totals.zip done
zbp14totals.zip done


In [10]:
nyc.head()

,ZIPCODE,SHAPE_AREA,SHAPE_LEN,geometry,zbp94,zbp95,zbp96,zbp97,zbp98,zbp99,...,zbp05,zbp06,zbp07,zbp08,zbp09,zbp10,zbp11,zbp12,zbp13,zbp14
0,11436,0.0,0.0,"POLYGON ((1038098.251871482 188138.3800067157,...",51,50,53,51,51,58,...,82,80,84,90,92,98,101,95,99,109
1,11213,0.0,0.0,"POLYGON ((1001613.712964058 186926.4395172149,...",406,420,433,456,476,473,...,556,565,556,592,621,675,674,726,755,772
2,11212,0.0,0.0,"POLYGON ((1011174.275535807 183696.33770971, 1...",505,494,502,527,519,541,...,587,570,587,602,647,699,713,721,731,752
3,11225,0.0,0.0,"POLYGON ((995908.3654508889 183617.6128015518,...",381,387,388,410,416,409,...,491,499,514,520,543,580,597,623,673,696
4,11218,0.0,0.0,"POLYGON ((991997.1134308875 176307.4958601296,...",985,1015,1049,1075,1118,1149,...,1332,1331,1377,1397,1457,1474,1523,1605,1687,1736


# Data cleaning

you may need to clean your data: for some NYC zip codes there may be no info
sanity check: you should have 20 (N_timestamps) datapoints per time series and about 250 zipcodes (Nzipcodes)


IMPORTANT: we talked about the importance of "whitening" your data.
Whitenings decorrelates the data: it makes the features independent so that the data covariance matrix is the identity matrix.
Whitening your data in time series analysis is in most cases **wrong**: you are modifying your time behaviour. This is because of the strong correlation between features (two consecutive time stamps for the same observation, the same zip code here, are strongly correlated). Here instead you want to standardize your time series: subtract the mean and divide each time series (separately) by its standard deviation. As a sanity check (if you use skitlearn Kmeans or skitlearns kmeans2): you want your data array to be shaped Nzipcodes x Ntimestamps

mydata.shape should be (Nzipcodes, Ntimestamps)

mydata[i].std() shoould be 1 for all i in range(len(Nzipcodes))

mydata[i].mean() should be ~0 for all i in range(len(Nzipcodes))



## TASKS:
    
    1. get and prep your data.
    2. cluster the NUMBER OF ESTABLISHMENTS time series with K-means in **a few** clusters (as discussed there is no real good, sound way to decide what a good number is here. try a few options, keeping in mind a few is more than a couple, but i recommand you stay within the single digit numbers)
    3. plot the cluster centers (if you used K means those are the means of the clusters). you can plot for example the cluster centers overlayed on each time series (using the alpha channel to control the opacity in the plot may be helpful here).
    4. Use another clustering algorithm (of your choice)
    5. overlay your data on a NYC map: you can use shapefiles for the zip codes and different colors for different clusters
    6. Compare the results of the 2 algorithms
    7. attempt an interpretation. this is dangerous ground: clustering is an exploratory tool so you do not want to jump to conclusions because you see some clusters! but seeing structure in your data can inform your next moves as an investigator. 
    

### The map of the clusters may look something like this

In [ ]:
...

### Figure 3: 
cloropleth of  cluster centers for 5 k-means clusters of business patterns (number of businesses) at the zipcode level for NYC zipcodes: each color indicates a cluster. The business pattern time series are plotted at the top.

### or maybe like this, depending on which algorithm you use, and how you proceed to preprocess your data and how you cluster it. There is no one correct answer, but general trends should be retrieved.

In [ ]:
...

### Figure 9: 
As figures 3, 5, 7 for hierarchical agglomerative clustering in 7 clusters, with smoothed time series

### And if you use hierarchical clustering and make a dandrogram it may look like this:

In [ ]:
from PIL import Image
Image.open("dandrogram.png")